In [ ]:
#Notes to self: 
#Image count seems to work, but it's giving me a giant number... What is it really counting??
#Dr. Wu has a time series function but not for sentinel yet.
#Matplotlib is super easy to use once I can export some dang LISTS
#Jan, Feb 2016 are missing data; March is basically just clouds

In [ ]:
#TODO:
#I need to find code that will let me export NDRE values to a list
    #So I can make a time series in matplotlib
    #Also system:time_start isn't working for me
    #Check code editor image count against python image count; maybe that will guide me

In [45]:
import ee
import geemap
import os
import matplotlib as plt

In [46]:
#Create Map and center on Manauas EC Flux Tower coordinates
Map = geemap.Map(center = (-2.609097222, -60.20929722), zoom = 10)
#Create a point for EC Flux Tower
brma2_point = ee.Geometry.Point(-60.20929722, -2.609097222)
#Create a 0.5x0.5 deg region centered over the EC Flux Tower
brma2_region = ee.Geometry.Rectangle(-60.50, -2.85, -60.00, -2.35)
Map

Map(center=[-2.609097222, -60.20929722], controls=(WidgetControl(options=['position'], widget=HBox(children=(T…

In [47]:
#Add the polygon to the map
Map.addLayer(brma2_region, {}, 'Manauas Region')

In [48]:
#Formula for NDRE
def addNDRE(image):
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')
    return image.addBands(ndre)

#Set visual paramters for NDRE
ndre_params = ({'min': 0.0,'max': 1.0, 'palette' : 
               ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']})

In [49]:
s2 = (ee.ImageCollection('COPERNICUS/S2')
      .map(lambda image : image.clip(brma2_region))
     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 40)))

visParams = {'bands' : ['B4', 'B3', 'B2'], 'max': 3000}

In [50]:
#Create cloud mask; hasn't been implemented yet
def maskclouds(image):
        qa = image.select('QA60')
        cloud_mask = ee.Number(2).pow(10).int()
        mask = band_qa.bitwiseAnd(cloud_mask).eq(0)
        return image.updateMask(mask)

In [51]:
jan2016 = s2.filterDate('2016-01-01', '2016-01-31')
feb2016 = s2.filterDate('2016-02-01', '2016-02-29')
mar2016 = s2.filterDate('2016-03-01', '2016-03-31')
apr2016 = s2.filterDate('2016-04-01', '2016-04-30')
may2016 = s2.filterDate('2016-05-01', '2016-05-31')
jun2016 = s2.filterDate('2016-06-01', '2016-06-30')
jul2016 = s2.filterDate('2016-07-01', '2016-07-31')
aug2016 = s2.filterDate('2016-08-01', '2016-08-31')
sep2016 = s2.filterDate('2016-09-01', '2016-09-30')
oct2016 = s2.filterDate('2016-10-01', '2016-10-31')
nov2016 = s2.filterDate('2016-11-01', '2016-11-30')
dec2016 = s2.filterDate('2016-12-01', '2016-12-31')

In [52]:
allMonths = (jan2016
             .merge(feb2016)
             .merge(mar2016)
             .merge(apr2016)
             .merge(may2016)
             .merge(jun2016)
             .merge(jul2016)
             .merge(aug2016)
             .merge(sep2016)
             .merge(oct2016)
             .merge(nov2016)
             .merge(dec2016))

In [53]:
Map.addLayer(allMonths, visParams, 'Raw Monthly Collection')

In [29]:
allMonths_ndre = allMonths.map(addNDRE).select('NDRE')
Map.addLayer(allMonths_ndre, ndre_params, 'Monthly NDRE Collection')

In [30]:
#Put the EC Tower on top of previous layer
Map.addLayer(brma2_point, {}, 'Manauas EC Flux Location')

In [34]:
#Can only export one image
Median_NDRE = allMonths_ndre.median()

#Exported for good fun; way to set palette for export? Directory is hard coded.
out_dir = '/Users/morgansteckler/Desktop/'
filename = os.path.join(out_dir, 'jan2016.tif')
geemap.ee_export_image(Median_NDRE, filename = filename, region = brma2_region, scale = 90, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /Users/morgansteckler/Desktop/jan2016.tif


In [32]:
#This is a step in the right direction, I think. It at least works.
#range = jan2016.reduceColumns(ee.Reducer.minMax(), ["system:time_start"])
#print('Date range: ', str(ee.Date(range.get('min')).getInfo()), str(ee.Date(range.get('max')).getInfo())+'\n')